#Bitcoin Transactions with Python Bitcoinlib

In this notebook, you will create transactions in Bitcoin test network.

While writing corresponding python statements, you may want to read  the related chapter(s) in Mastering Bitcoin or the resources below. 

1. Bitcoin Script: https://en.bitcoin.it/wiki/Script
2. Bitcoin Transaction Format: https://en.bitcoin.it/wiki/Transaction
3. Python-bitcoin docs: https://media.readthedocs.org/pdf/python-bitcoinlib/latest/python-bitcoinlib.pdf

Lets first install **python-bitcoinlib** which we will use to connect to the Bitcoin network and to generate a public-private key pair.

In [0]:
!pip install python-bitcoinlib
!pip install requests

     |████████████████████████████████| 102kB 3.8MB/s 


Lets create a Bitcoin address and accompanying private key for ourselves with the code below. 

In [0]:
from os import urandom
from bitcoin import SelectParams
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress

SelectParams('testnet')

seckey = CBitcoinSecret.from_secret_bytes(urandom(32))

print("Private key: %s" % seckey)
print("Address: %s" %P2PKHBitcoinAddress.from_pubkey(seckey.pub))

Private key: cQh9SATDjNdPhJioC7bghbyQmNzvHA9MtViY7W3Rg1TiGPNuU3PZ
Address: mo2F93cvEtRafTCbuTiXv9D13nJJ9KBaWi


Now with this address, we can get free testnet coins from the coin faucet. Click [here](https://coinfaucet.eu/en/btc-testnet/) to get some free coins on the *Bitcoin test network*. Note that it is a nice thing to send back the bitcoin's you got to the faucet. 

Once you click the button, a transaction, which is a P2PKH transaction, will be generated and sent to the Bitcoin Test network (by the faucet). One of the outputs of this transaction will be for your Public Key Hash. That output will be our main UTXO to spend. 

Here is a sample output. Since your address is different your coins will be sent to a different address.

```
We sent 0.12133031 bitcoins to address
mnjD9piZu9DTkoeCMGPQSookMTqL7DBPod

tx: ae2d8d7d78fb0c56582a76816a5dcdda14f945413b49ef97455b5fbf5d952be0
Send coins back, when you don't need them anymore to the address

mv4rnyY3Su5gjcDNzbMLKBQkBicCtHUtFB```

You can see the transaction details by checking all the transactions for your account by typing the correct address in this [web-page](https://live.blockcypher.com/btc-testnet/address/mnjD9piZu9DTkoeCMGPQSookMTqL7DBPod/).

Before continuing, lets configure our Python environment with our public and private key pair. This is nothing fancy; we just set some variables' values to be used later in the codes and the exercises in the rest of this notebook. 

Note that we will use the same bitcoin address, the one we created above, to correctly configure our environment. 

In [0]:
from bitcoin import SelectParams
from bitcoin.base58 import decode
from bitcoin.wallet import CBitcoinAddress, CBitcoinSecret, P2PKHBitcoinAddress

#This says that we will connect to testnet, not the actual Bitcoin network
SelectParams('testnet')

#This is my private key. You need to add yours here
my_private_key = CBitcoinSecret('cQh9SATDjNdPhJioC7bghbyQmNzvHA9MtViY7W3Rg1TiGPNuU3PZ') 

#As you see, we do not explicitly see the public key  anywhere. We do not need to see it. 
#The elliptic curve operations are handled in the background. Your private key
#is in fact a scalar which will be multiplied with a point on the curve to obtain
#the public key (note: elliptic curve discrete logarithm problem).
my_public_key = my_private_key.pub 

#With P2PKHBitcoinAddress's from_pubkey function, we can obtain our address. 
my_address = P2PKHBitcoinAddress.from_pubkey(my_public_key)

#This is the address of the faucet (a Bitcoin Test account) which we will 
#send our money back during/after the exercises. 
faucet_address = CBitcoinAddress('mv4rnyY3Su5gjcDNzbMLKBQkBicCtHUtFB') 

print("Private key: %s" % my_private_key)
print("Address: %s" %P2PKHBitcoinAddress.from_pubkey(my_private_key.pub))

Private key: cQh9SATDjNdPhJioC7bghbyQmNzvHA9MtViY7W3Rg1TiGPNuU3PZ
Address: mo2F93cvEtRafTCbuTiXv9D13nJJ9KBaWi


The first thing we will do will be splitting the UTXO obtained from the faucet into multiple UTXOs. We may do mistakes, spend an UTXO in an unlockable manner and lose some of our test coins. So it is a good exercise to split the only UTXO we have.  

Lets run the **split_coins** function below with the transaction we obtained from the faucet.  The code is given below: You do not need to change the **split_coins** function. However, you need to add the UTXO information we obtained from the faucet at the end.

You can check the status of the outputs in the original faucet transaction from BlockCypher (as I do it [here](https://live.blockcypher.com/btc-testnet/tx/1adee36659bc7268d33b3d90ebc7c7cf807f363f11a572076b3a3aa9845342d1/)). Before running the code below it must be *unspent*. Read the code and try to understand what it does by matching what we have learned in the class with the statements below.

If you do everything correctly, at the end, you need to get a response with code 201. That is, you will see something like 
```
201 Created
{
  "tx": {
        ....
  }
}
 ```
at the beginning of the output. And after some time, depending on the fee you give to the miners, the UTXO you used will appear as spent. Fortunately,  you have many more now to be used in the exercises. 

In [0]:
import requests

from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH

def create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, seckey):
    #We want no parts of the transaction to be changed.
    #Hence, we hash all the transaction before signing it (i.e.: SIGHASH_ALL). 
    sighash = SignatureHash(txin_scriptPubKey, tx, 0, SIGHASH_ALL)
    signature = seckey.sign(sighash) + bytes([SIGHASH_ALL])
    return signature

def broadcast_transaction(tx):
    raw_transaction = b2x(tx.serialize())
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    return requests.post(
        'https://api.blockcypher.com/v1/btc/test3/txs/push',
        headers=headers,
        data='{"tx": "%s"}' % raw_transaction)

#What we simply do in this transaction is splitting. We spent the faucet output and
#split the amount into n pieces each is again to our addresses in a P2PKH structure.
def split_coins(amount_to_send, txid_to_spend, utxo_index, n):
    #We are creating a transaction with a single input and n outputs.
    
    #txin is the input part of the transaction 
    txin_scriptPubKey = my_address.to_scriptPubKey() #initial UTXO was for my address so we need to use it as scriptPubKey 
    txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index)) #now we have txin
    #The function lx() onverts a little-endian hex string to bytes
    #The COutPoint is the combination of a transaction hash and an index n into its vout as we have discussed in the lecture
    
    #This transaction's outputs are all for myself - so the default P2PKH script
    #[OP_DUP OP_HASH160 <pubKeyHash> OP_EQUALVERIFY OP_CHECKSIG] must be the scriptPubKey
    #The function to_scriptPubKey automatically generates this script from a P2PKHBitcoinAddress
    #Note that the script contains a base16 encoded version of checksummed/modified byte58 encoded address (This operation is reversible!).
    txout_scriptPubKey = my_address.to_scriptPubKey() 
    
    #txout is a single output. (IMP: the function CScript which you use for the first time serializes the script created above)
    txout = CMutableTxOut((amount_to_send / n) * COIN, CScript(txout_scriptPubKey)) #amount and address are given here

    #This is the whole transaction we will broadcast
    #Note the [txout]*n in the parameters of CMutableTransaction function
    tx = CMutableTransaction([txin], [txout] * n)
    
    #Now lets sign the transaction to keep its integrity
    sig = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, my_private_key)
    #We need to create the unlocking script which is [signature, encoded SIGHASH_ALL] + PublicKey
    txin.scriptSig = CScript([sig, my_public_key])  
    
    #Lets validate if the script is correctly generated... 
    #This function raises a ValidationError subclass if the validation fails
    VerifyScript(txin.scriptSig, txin_scriptPubKey, tx, 0, (SCRIPT_VERIFY_P2SH,))
   
    #Broadcast the transaction
    response = broadcast_transaction(tx)
    print(response.status_code, response.reason)
    print(response.text)

#The mount of BTC in the output you're splitting minus fee.
#Warning: In the test network, fees are required. They can be small but since this
#is an exercise, you can be generous
amount_to_send = 0.036 

#This must be the transaction id you have from the faucet's transaction
txid_to_spend = ('7af130117949e948972b2118a4e8bd40762a58e6b5ab31513363370e2335cb95')

#the output UTXO location in the transaction from faucet (mine is 1, but maybe yours is 0)
utxo_index = 1 

#number of transactions to be splitted
n = 12 
######################################################################

split_coins(amount_to_send, txid_to_spend, utxo_index, n)

201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "4cce54daf7f92da317b5b3f42900410326f51358b91139542f54aec6c2e651a1",
    "addresses": [
      "mo2F93cvEtRafTCbuTiXv9D13nJJ9KBaWi"
    ],
    "total": 3599988,
    "fees": 34394,
    "size": 566,
    "preference": "high",
    "relayed_by": "35.222.162.148",
    "received": "2019-11-30T09:17:41.52387281Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 12,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "7af130117949e948972b2118a4e8bd40762a58e6b5ab31513363370e2335cb95",
        "output_index": 1,
        "script": "483045022100c8e8731f9b25003747fc6d8d35f35f79fcfab7c07ae059b53ddf981d386f73c30220617cca2392f4468f7ecd9106b210a2bb716e980a27a84fedd5b805e80a0fd71201210327f738cd5b5ef73db8456caec9f6e0794051ff97ab1afaa94545ead83df5bef1",
        "output_value": 3634382,
        "sequence": 4294967295,
        "addresses": [
          "mo2F93cvEtRafTCbuTiXv9D13nJJ9KBaWi"

Lets spend some of the small UTXOs and send it back to the **faucet**. Make it complete, run, and be sure that you have the correct output with code *201 Created*.


In [0]:
from bitcoin.core.script import *

# TODO: Complete this script to unlock the BTC that was sent to you
# in the PayToPublicKeyHash transaction. You may need to use variables
# that are globally defined.
def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey):
    txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index))
    txout = CMutableTxOut(amount_to_send * COIN, CScript(txout_scriptPubKey))
    tx = CMutableTransaction([txin], [txout])

    txin_scriptPubKey = my_address.to_scriptPubKey()
    signature = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, my_private_key);
    txin.scriptSig = CScript([signature, my_public_key])
    
    VerifyScript(txin.scriptSig, txin_scriptPubKey, tx, 0, (SCRIPT_VERIFY_P2SH,))

    return broadcast_transaction(tx)
  
######################################################################
# TODO: set all these parameters correctly
amount_to_send = 0.0005 #Do not forget the fee
txid_to_spend = ('58d6e7f2cc881870b0adf94a163dde06c37cc489d184e72e389c53c5c0eb2528')
utxo_index = 0
txout_scriptPubKey = faucet_address.to_scriptPubKey() #[OP_DUP ....]
######################################################################

response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "7d92b16ba15849517f8acd64527aa52fbfb8353a7836e15bdb55f2be26ca8185",
    "addresses": [
      "mpPD7cDGdyLLD8BkS18wW1Xwmp9w1cNEGE",
      "2N4XgqsKhMPL4n6tJoPVGHPNEgsphWnXoFt"
    ],
    "total": 50000,
    "fees": 249999,
    "size": 190,
    "preference": "high",
    "relayed_by": "35.227.99.86",
    "received": "2019-11-25T06:59:40.368248489Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "58d6e7f2cc881870b0adf94a163dde06c37cc489d184e72e389c53c5c0eb2528",
        "output_index": 0,
        "script": "483045022100859c24fc66ce92378730b28628dfe7576426653e396b1968a1f3e9e3ef0f05b002204f51aa290a95fb4ff51be81ba22374e5b3f506d6bbcba6388e6ff6b3aa2140e70121029285c2c64cf75ba712860ff4a423607d428512fb084fbdae81d3d80f8ab8bdad",
        "output_value": 299999,
        "sequence": 4294967295,
        "addresses": [
     

Now you know how to create a P2PKH transaction. 